In [1]:
import pygame
from pygame.locals import *
import numpy as np
import numpy.random as rnd
from matplotlib import pyplot as plt

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Environment():
    
    def __init__(self,num_obstacles,num_greenbeans=1):
        self.screen = pygame.display.set_mode((1200,600))
        self.clock = pygame.time.Clock()
        
        self.obstacles = []
        for k in range(num_obstacles):
            crect = pygame.rect.Rect(rnd.randint(50,1100,1),rnd.randint(50,500,1),rnd.randint(20,100,1),rnd.randint(20,100,1))
            self.obstacles.append(crect)
        
    def draw_env(self):
        self.screen.fill(0)
        for obstacle in self.obstacles:
            pygame.draw.rect(self.screen,(0,255,0),obstacle)
        
    def draw_player(self,agent):
        brightness = 255
        pygame.draw.rect(self.screen,(0,0,brightness),agent.rect)
        pygame.draw.rect(self.screen,(255,255,255),agent.rect_t)
        pygame.draw.rect(self.screen,(255,255,255),agent.rect_r)
        pygame.draw.rect(self.screen,(255,255,255),agent.rect_l)
        pygame.draw.rect(self.screen,(255,255,255),agent.rect_b)

In [3]:
class Agent():
    
    def __init__(self,pos_init=(200,100),size=(20,20),maxstep=5,spots_offset=5,num_obstacles=5):
        
        self.pos = pos_init
        self.maxstep = maxstep
        self.size = size
        self.spots_offset = spots_offset
        self.pain_t = 0
        self.pain_r = 0
        self.pain_l = 0
        self.pain_b = 0
        
        self.dir = [0,0]
        
        self.rect = pygame.rect.Rect((self.pos[0],self.pos[1],self.size[0],self.size[1]))
        
        self.rect_t = pygame.rect.Rect((self.pos[0]+self.spots_offset,self.pos[1],self.size[0]-2*self.spots_offset,1))
        self.rect_r = pygame.rect.Rect((self.pos[0]+self.size[0]-1,self.pos[1]+self.spots_offset,1,self.size[1]-2*self.spots_offset))
        self.rect_l = pygame.rect.Rect((self.pos[0],self.pos[1]+self.spots_offset,1,self.size[1]-2*self.spots_offset))
        self.rect_b = pygame.rect.Rect((self.pos[0]+self.spots_offset,self.pos[1]+self.size[1]-1,self.size[0]-2*self.spots_offset,1))
        
    def get_pos(self):
        return (self.rect[0],self.rect[1])
    
    def stop(self):
        self.dir = [0,0]
        
    def softmax(self,x):
        return np.exp(x-np.max(x))/np.sum(np.exp(x-np.max(x)))
        
    def accelerate(self,vvec):
        newdir = [self.dir[0]+vvec[0],self.dir[1]+vvec[1]]
        if newdir[0]<=self.maxstep and newdir[0]>=-self.maxstep:
            self.dir[0] = newdir[0]
        if newdir[1]<=self.maxstep and newdir[1]>=-self.maxstep:
            self.dir[1] = newdir[1]
        
    def move(self,env):
        screen_rect = env.screen.get_rect()
            
        self.pain_r = False
        self.pain_l = False
        self.pain_b = False
        if any([self.rect_t.colliderect(o) for o in env.obstacles]) or self.rect_t[1]==0:
            self.pain_t += 1
        else:
            if self.pain_t>0:
                self.pain_t -= 1
        if any([self.rect_r.colliderect(o) for o in env.obstacles]) or self.rect_r[0]+1==1200:
            self.pain_r += 1
        else:
            if self.pain_r>0:
                self.pain_r -= 1
        if any([self.rect_l.colliderect(o) for o in env.obstacles]) or self.rect_l[0]==0:
            self.pain_l += 1
        else:
            if self.pain_l>0:
                self.pain_l -= 1
        if any([self.rect_b.colliderect(o) for o in env.obstacles]) or self.rect_b[1]+1==600:
            self.pain_b += 1
        else:
            if self.pain_b>0:
                self.pain_b -= 1
        
        self.decide_action(env)
        
        self.rect.move_ip(self.dir[0],0)
        self.rect.move_ip(0,self.dir[1])
        self.rect.clamp_ip(screen_rect)
        
        self.pos = (self.rect[0],self.rect[1]) 
        
        self.rect_t = pygame.rect.Rect((self.pos[0]+self.spots_offset,self.pos[1],self.size[0]-2*self.spots_offset,1))
        self.rect_r = pygame.rect.Rect((self.pos[0]+self.size[0]-1,self.pos[1]+self.spots_offset,1,self.size[1]-2*self.spots_offset))
        self.rect_l = pygame.rect.Rect((self.pos[0],self.pos[1]+self.spots_offset,1,self.size[1]-2*self.spots_offset))
        self.rect_b = pygame.rect.Rect((self.pos[0]+self.spots_offset,self.pos[1]+self.size[1]-1,self.size[0]-2*self.spots_offset,1))
   
    def collideswith_screenedge(self):
        position = self.get_pos()
        if position[0]==0 or position[0]==1200-self.size[0]:
            return True
        if position[1]==0 or position[1]==600-self.size[1]:
            return True
        return False
    
    def update(self):
        # START TODO
        
        # STOP TODO
        return
    
    def think(self):
        m_qual = np.zeros(shape=[4,])
        if self.pain_t:
            self.stop()
            self.update()
            m_qual[0]=1
            return m_qual
        elif self.pain_r:
            self.stop()
            self.update()
            m_qual[1]=1
            return m_qual
        elif self.pain_l:
            self.stop()
            self.update()
            m_qual[2]=1
            return m_qual
        elif self.pain_b:
            self.stop()
            self.update()
            m_qual[3]=1
            return m_qual
        
        # START TODO
        m_qual[rnd.randint(4)]=1
        # STOP TODO
        
        return m_qual

    def decide_action(self,env):
        
        m_qual = self.think()
            
        m_qual = self.softmax(m_qual)
        if np.argmax(m_qual)==0:
            self.accelerate([0,1])
        if np.argmax(m_qual)==1:
            self.accelerate([-1,0])
        if np.argmax(m_qual)==2:
            self.accelerate([1,0])
        if np.argmax(m_qual)==3:
            self.accelerate([0,-1])

In [5]:
num_obstacles = 10

pygame.init()
env = Environment(num_obstacles)
player = Agent()

try:
    
    running = True
    while running==True:
        
        env.draw_env()
        env.draw_player(player)
        player.move(env)
        pygame.display.flip()
        
        if any([e.type==pygame.KEYDOWN for e in pygame.event.get()]):
            running=False
        
        env.clock.tick(100)
    pygame.quit()
except SystemExit:
    pygame.quit()